In [2]:
import pandas as pd
from sklearn import feature_selection

In [24]:
df_modelo = pd.read_csv('df_modelo')

In [35]:
df_modelo[df_modelo['resultado'].isna()]

,ano,rodata,vencedor,aproveitamento,cartao_amarelo,cartao_vermelho,chutes,chutes_no_alvo,escanteios,faltas,impedimentos,passes,posse_de_bola,precisao_passes,resultado
30,2018,6,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31,2018,6,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,2018,6,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33,2018,6,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34,2018,6,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35,2018,6,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,2018,6,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1673,2020,12,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1674,2020,12,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1675,2020,12,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
X = df_modelo
y = df_modelo.reset_index()['resultado']

mic = feature_selection.mutual_info_classif(X, y, random_state = 1333)

fig, ax = plt.subplots(figsize=(10, 8))

features = pd.DataFrame({
    'feature': X.columns,
    "vimp": mic
})

features.set_index('feature').sort_values(by = 'vimp').plot.barh(ax = ax)

C:\Users\henri\anaconda3\Lib\site-packages\sklearn\utils\_array_api.py:245: RuntimeWarning: invalid value encountered in cast
  return x.astype(dtype, copy=copy, casting=casting)


ValueError: Input y contains NaN.